In [69]:
import pyspark
import pandas as pd 
from pyspark.sql import SparkSession 
from pyspark.context import SparkContext
from pyspark.sql import functions
from pyspark.sql.types import * 





In [3]:
spark = SparkSession.builder.master("local")\
    .appName("Airline_data")\
    .getOrCreate()

In [33]:
df = spark.read.csv("Airline.csv", header=True, inferSchema=True)

In [67]:
df.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Quarter: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- FlightDate: string (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- Carrier: string (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- Origin: string (nullable = true)
 |-- OriginCityName: string (nullable = true)
 |-- OriginState: string (nullable = true)
 |-- OriginStateFips: integer (nullable = true)
 |-- OriginStateName: string (nullable = true)
 |-- OriginWac: integer (nullable = true)
 |-- Dest: string (nullable = true)
 |-- DestCityName: string (nullable = true)
 |-- DestState: string (nullable = true)
 |-- DestStateFips: integer (nullable = true)
 |-- DestStateName: string (nullable = true)
 |-- DestWac: integer (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- DepTime: string (nullabl

In [35]:
print(df.count(),len(df.columns))


5000 83


In [52]:
ids = [i for i in df.columns if "ID" in i]
print("rimuovere : ", ids)

rimuovere :  ['AirlineID', 'OriginAirportID', 'OriginAirportSeqID', 'OriginCityMarketID', 'DestAirportID', 'DestAirportSeqID', 'DestCityMarketID', 'Div1AirportID', 'Div1AirportSeqID', 'Div2AirportID', 'Div2AirportSeqID']


In [58]:
df = df.drop(*ids)
print(len(df.columns))

72


In [65]:
to_int = ["DestStateFips", "DestWac","CRSDepTime","DepDel15","ArrDel15","Cancelled","Flights"]
for i in to_int: 
    df = df.withColumn(i, col(i).cast('int'))


In [66]:
to_float = ["DepDelay", "TaxiOut","WheelsOn","TaxiIn","CRSElapsedTime"]
for i in to_float: 
    df = df.withColumn(i, col(i).cast('float'))


In [89]:
df_num = df.select(to_int + to_float)

In [90]:
df_num.printSchema()

root
 |-- DestStateFips: integer (nullable = true)
 |-- DestWac: integer (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- DepDel15: integer (nullable = true)
 |-- ArrDel15: integer (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- Flights: integer (nullable = true)
 |-- DepDelay: float (nullable = true)
 |-- TaxiOut: float (nullable = true)
 |-- WheelsOn: float (nullable = true)
 |-- TaxiIn: float (nullable = true)
 |-- CRSElapsedTime: float (nullable = true)



In [120]:
df_num.show(100)

+-------------+-------+----------+--------+--------+---------+-------+--------+-------+--------+------+--------------+
|DestStateFips|DestWac|CRSDepTime|DepDel15|ArrDel15|Cancelled|Flights|DepDelay|TaxiOut|WheelsOn|TaxiIn|CRSElapsedTime|
+-------------+-------+----------+--------+--------+---------+-------+--------+-------+--------+------+--------------+
|           26|     43|      1100|       0|       0|        0|      1|    -3.0|   15.0|  1424.0|   8.0|         158.0|
|           26|     43|      1100|       0|       0|        0|      1|    -4.0|   14.0|  1416.0|  10.0|         158.0|
|           26|     43|      1100|       0|       0|        0|      1|    -5.0|   21.0|  1431.0|  14.0|         158.0|
|           26|     43|      1100|       0|       0|        0|      1|     2.0|   13.0|  1424.0|   9.0|         158.0|
|           26|     43|      1100|       1|       1|        0|      1|   100.0|   20.0|  1617.0|  14.0|         158.0|
|           26|     43|      1100|       0|     

In [121]:
df_num = df_num.na.fill(0)

In [105]:
toDrop = []
for i in range(len(df_num.columns)):
    for j in range(i) :
       # print(df_num.columns[i], df_num.columns[j])
       # print(df_num.stat.corr(df_num.columns[i], df_num.columns[j]))
        if abs(df_num.stat.corr(df_num.columns[i], df_num.columns[j]))> 0.8:
               toDrop.append(df_num.columns[i])
print(toDrop)

[]


In [133]:
to_int  + to_float

['DestStateFips',
 'DestWac',
 'CRSDepTime',
 'DepDel15',
 'ArrDel15',
 'Cancelled',
 'Flights',
 'DepDelay',
 'TaxiOut',
 'WheelsOn',
 'TaxiIn',
 'CRSElapsedTime']

In [144]:
from pyspark.ml.feature  import VectorAssembler
assembler = VectorAssembler().setInputCols(['DestStateFips','DestWac','CRSDepTime','DepDel15',\
                                           'Cancelled','Flights','DepDelay','TaxiOut','WheelsOn','TaxiIn','CRSElapsedTime'])\
.setOutputCol("features")

df_num = assembler.transform(df_num)
df_num.show(5)

+-------------+-------+----------+--------+--------+---------+-------+--------+-------+--------+------+--------------+--------------------+
|DestStateFips|DestWac|CRSDepTime|DepDel15|ArrDel15|Cancelled|Flights|DepDelay|TaxiOut|WheelsOn|TaxiIn|CRSElapsedTime|            features|
+-------------+-------+----------+--------+--------+---------+-------+--------+-------+--------+------+--------------+--------------------+
|           26|     43|      1100|       0|       0|        0|      1|    -3.0|   15.0|  1424.0|   8.0|         158.0|[26.0,43.0,1100.0...|
|           26|     43|      1100|       0|       0|        0|      1|    -4.0|   14.0|  1416.0|  10.0|         158.0|[26.0,43.0,1100.0...|
|           26|     43|      1100|       0|       0|        0|      1|    -5.0|   21.0|  1431.0|  14.0|         158.0|[26.0,43.0,1100.0...|
|           26|     43|      1100|       0|       0|        0|      1|     2.0|   13.0|  1424.0|   9.0|         158.0|[26.0,43.0,1100.0...|
|           26|     

In [152]:

train,test = df_num.randomSplit([0.7,0.3], seed=900)

In [153]:
train.count()

3486

In [171]:
from pyspark.ml.classification import LogisticRegression 
import random as rnd
acc=[]
for i in range(10):
    train,test = df_num.randomSplit([0.7,0.3], seed=rnd.randint(1,300))
    logit = LogisticRegression(featuresCol = "features", labelCol = "ArrDel15")
    model = logit.fit(train)
    pred = model.transform(test)
    accuracy = pred.filter(pred.ArrDel15 == pred.prediction).count() / pred.count()
    acc.append(round(accuracy,3))
print(acc)

[0.93, 0.922, 0.937, 0.936, 0.94, 0.933, 0.931, 0.928, 0.929, 0.935]
